In this script we want to try to:
    - use the trained generator to extract the latent representation
    
Taking inspiration from here https://github.com/spiorf/stylegan-encoder/blob/master/encode_images.py

In [11]:
import os
import sys
sys.path.append('/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/stylegan')
sys.path.append('/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/stylegan-encoder')
import pickle
import numpy as np
from tqdm import tqdm
import dnnlib
import config
import dnnlib.tflib as tflib
from encoder.generator_model import Generator
from encoder.perceptual_model import PerceptualModel

%matplotlib inline
PATH_GAN = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/results_styleGAN/00005-sgan-sdo-4gpu/network-snapshot-008040.pkl'
PATH_PLOTS = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/results_styleGAN/plots'
src_dir = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/datasets_styleGAN/sdo'
generated_images_dir = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/results_styleGAN/generated_images'
dlatent_dir = '/gpfs/gpfs_gl4_16mb/b9p111/b9p111ap/results_styleGAN/dlatent'

In [13]:
def split_to_batches(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [12]:
ref_images = [os.path.join(src_dir, x) for x in os.listdir(src_dir)]
ref_images = list(filter(os.path.isfile, ref_images))

if len(ref_images) == 0:
    raise Exception('%s is empty' % args.src_dir)

os.makedirs(generated_images_dir, exist_ok=True)
os.makedirs(dlatent_dir, exist_ok=True)

In [15]:
tflib.init_tf()
batch_size = 1

with open(PATH_GAN, 'rb') as pickle_file:
    _, _, Gs_network = pickle.load(pickle_file)
    generator = Generator(Gs_network, batch_size)
    generated_images = generator.generate_images()
    
    generator = Generator(Gs_network, batch_size, randomize_noise=False)
    perceptual_model = PerceptualModel(512, layer=9, batch_size=batch_size)
    perceptual_model.build_perceptual_model(generator.generated_image)

    # Optimize (only) dlatents by minimizing perceptual loss between reference and generated images in feature space
    for images_batch in tqdm(split_to_batches(ref_images, args.batch_size), total=len(ref_images)//batch_size):
        names = [os.path.splitext(os.path.basename(x))[0] for x in images_batch]

        perceptual_model.set_reference_images(images_batch)
        op = perceptual_model.optimize(generator.dlatent_variable)
        pbar = tqdm(op)
        for loss in pbar:
            pbar.set_description(' '.join(names)+' Loss: %.2f' % loss)
        print(' '.join(names), ' loss:', loss)

        # Generate images from found dlatents and save them
        generated_images = generator.generate_images()
        generated_dlatents = generator.get_dlatents()
        for img_array, dlatent, img_name in zip(generated_images, generated_dlatents, names):
            img = PIL.Image.fromarray(img_array, 'RGB')
            img.save(os.path.join(args.generated_images_dir, f'{img_name}.png'), 'PNG')
            np.save(os.path.join(args.dlatent_dir, f'{img_name}.npy'), dlatent)

        generator.reset_dlatents()


StopIteration: 